### *Install and Import Libraries*

In [ ]:
%pip install -q "monai[nibabel, tqdm]"
!pip install itk
!pip install itkwidgets

In [48]:
import itk
import itkwidgets
import numpy as np
import nibabel as nib
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

import os
import glob
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import monai
from monai.data import Dataset, CacheDataset, DataLoader
from monai.losses import DiceLoss
from monai.metrics import compute_meandice
from monai.inferers import sliding_window_inference
from monai.transforms import (
    AddChanneld,
    AsDiscrete,
    CastToTyped,
    LoadNiftid,
    Orientationd,
    RandAffined,
    RandCropByPosNegLabeld,
    RandFlipd,
    RandGaussianNoised,
    ScaleIntensityRanged,
    Spacingd,
    SpatialPadd,
    ToTensord,
)
from monai.transforms import Compose

%matplotlib inline
sns.set_style('darkgrid')

In [49]:
keys = ('image','label')

### *Load Data*

In [50]:
# Load Data and transforms

DIR = './COVID-19-20/Train'

images = sorted(glob.glob(os.path.join(DIR, 'volume-covid19-A-0011_ct.nii.gz')))
labels = sorted(glob.glob(os.path.join(DIR, 'volume-covid19-A-0011_seg.nii.gz')))
data = [{keys[0]: img, keys[1]: seg} for img,seg in zip(images, labels)]

### *Before Transformations*

In [51]:
image_data = nib.load("./COVID-19-20/Train/volume-covid19-A-0011_ct.nii.gz")
image_data = image_data.get_fdata()
label_data = nib.load("./COVID-19-20/Train/volume-covid19-A-0011_seg.nii.gz")
label_data = label_data.get_fdata()

print("Shape before transformation:")
print('Image:', image_data.shape)
print('Label:', label_data.shape)

Shape before transformation:
Image: (512, 512, 296)
Label: (512, 512, 296)


In [52]:
def explore_img(layer):
    plt.figure(figsize=(10, 5))
    plt.imshow(image_data[:, :, layer], cmap='gray');
    plt.title('Explore Layers of Brain MRI', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_img, layer=(0, image_data.shape[2] - 1));

interactive(children=(IntSlider(value=147, description='layer', max=295), Output()), _dom_classes=('widget-int…

In [53]:
def explore_img(layer):
    plt.figure(figsize=(10, 5))
    plt.imshow(label_data[:, :, layer], cmap='gray');
    plt.title('Explore Layers of Brain MRI', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_img, layer=(0, label_data.shape[2] - 1));

interactive(children=(IntSlider(value=147, description='layer', max=295), Output()), _dom_classes=('widget-int…

### *Transformations*

#### Load data, Add_channel dimension, orientation

In [39]:
trans = Compose([LoadNiftid(keys),
                 AddChanneld(keys),
                 Orientationd(keys, axcodes = "LPS"),
                 CastToTyped(keys, dtype=(np.float32, np.uint8)),
                 ToTensord(keys)
                 ])

In [40]:
dataset = Dataset(data = data,
                  transform = trans)

print('Image:', dataset[0]['image'].shape)
print('Label:', dataset[0]['label'].shape)

Image: torch.Size([1, 512, 512, 296])
Label: torch.Size([1, 512, 512, 296])


In [23]:
def explore_img(layer):
    plt.figure(figsize=(10, 5))
    plt.imshow(dataset[0]['image'].squeeze()[:, :, layer], cmap='gray');
    plt.title('Explore Layers', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_img, layer=(0, dataset[0]['image'].shape[3] - 1));

interactive(children=(IntSlider(value=147, description='layer', max=295), Output()), _dom_classes=('widget-int…

In [24]:
def explore_img(layer):
    plt.figure(figsize=(10, 5))
    plt.imshow(dataset[0]['label'].squeeze()[:, :, layer], cmap='gray');
    plt.title('Explore Layers', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_img, layer=(0, dataset[0]['label'].shape[3] - 1));

interactive(children=(IntSlider(value=147, description='layer', max=295), Output()), _dom_classes=('widget-int…

#### Spacingd

In [45]:
trans = Compose([LoadNiftid(keys),
                 AddChanneld(keys),
                 Orientationd(keys, axcodes = "LPS"),
                 Spacingd(keys, pixdim=(1.0, 1.0, 5.0), mode=("bilinear", "nearest")[: len(keys)]),
                 CastToTyped(keys, dtype=(np.float32, np.uint8)),
                 ToTensord(keys)
                 ])

In [ ]:
dataset = Dataset(data = data,
                  transform = trans)

print('Image:', dataset[0]['image'].shape)
print('Label:', dataset[0]['label'].shape)

In [32]:
def explore_img(layer):
    plt.figure(figsize=(10, 5))
    plt.imshow(dataset[0]['image'].squeeze()[:, :, layer], cmap='gray');
    plt.title('Explore Layers', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_img, layer=(0, dataset[0]['image'].shape[3] - 1));

interactive(children=(IntSlider(value=29, description='layer', max=59), Output()), _dom_classes=('widget-inter…

In [33]:
def explore_img(layer):
    plt.figure(figsize=(10, 5))
    plt.imshow(dataset[0]['label'].squeeze()[:, :, layer], cmap='gray');
    plt.title('Explore Layers', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_img, layer=(0, dataset[0]['label'].shape[3] - 1));

interactive(children=(IntSlider(value=29, description='layer', max=59), Output()), _dom_classes=('widget-inter…

#### SpatialPadd, RandCropByPosNegLabeld, RandAffined, RandGaussianNoised

In [73]:
trans = Compose([LoadNiftid(keys),
                 AddChanneld(keys),
                 Orientationd(keys, axcodes = "LPS"),
                 Spacingd(keys, pixdim=(1.0, 1.0, 5.0), mode=("bilinear", "nearest")[: len(keys)]),
                 SpatialPadd(keys, spatial_size=(192, 192, -1), mode="reflect"),          # ensure at least 192x192
                 RandAffined(
                    keys,
                    prob=0.15,
                    rotate_range=(-0.05, 0.05),
                    scale_range=(-0.1, 0.1),
                    mode=("bilinear", "nearest"),
                    as_tensor_output=False),
                 RandCropByPosNegLabeld(keys, label_key=keys[1], spatial_size=(192, 192, 16), num_samples=3),
                 RandGaussianNoised(keys[0], prob=0.15, std=0.01),
                 CastToTyped(keys, dtype=(np.float32, np.uint8)),
                 ToTensord(keys)
                 ])

In [74]:
dataset = Dataset(data = data,
                  transform = trans)

print('Image:', dataset[0][0]['image'].shape)
print('Label:', dataset[0][0]['label'].shape)

Image: torch.Size([1, 192, 192, 16])
Label: torch.Size([1, 192, 192, 16])


In [75]:
def explore_img(layer):
    plt.figure(figsize=(10, 5))
    plt.imshow(dataset[0][0]['image'].squeeze()[:, :, layer], cmap='gray');
    plt.title('Explore Layers', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_img, layer=(0, dataset[0][0]['image'].shape[3] - 1));

interactive(children=(IntSlider(value=7, description='layer', max=15), Output()), _dom_classes=('widget-intera…

In [76]:
def explore_img(layer):
    plt.figure(figsize=(10, 5))
    plt.imshow(dataset[0][0]['label'].squeeze()[:, :, layer], cmap='gray');
    plt.title('Explore Layers', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_img, layer=(0, dataset[0][0]['label'].shape[3] - 1));

interactive(children=(IntSlider(value=7, description='layer', max=15), Output()), _dom_classes=('widget-intera…